In [1]:
import spacy
nlp=spacy.load('en_core_web_sm')

nlp.pipe_names

['tagger', 'parser', 'ner']

In [2]:
# article_text = 'issue advertisements in an English and a Hindi newspaper having nationwide circulation and if so directed by the Recovery Officer, issue such advertisement in vernacular newspaper(s) having circulation in the area(s) where the investors are concentrated'
article_text = 'issue advertisements in an English and a Hindi newspaper having nationwide circulation and if so directed by the Recovery Officer, issue such advertisement in vernacular newspaper(s) having circulation in the area(s) where the investors are concentrated'

doc=nlp(article_text)
for ent in doc.ents:
  print(ent.text,ent.label_)

English LANGUAGE
Hindi GPE
vernacular newspaper(s GPE


In [3]:
ner=nlp.get_pipe("ner")

Ujwal's annotations

In [15]:
import pickle
with open('uj_train.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    uj_train = pickle.load(f)
print(uj_train)    

[('"NOTE: It is intended that disclosure of trades would need to be of not only those executed by the person concerned but also by the immediate relatives and of other persons for whom the person concerned takes trading decisions. These regulations are primarily aimed at preventing abuse by trading when in possession of unpublished price sensitive information and therefore, what matters is whether the person who takes trading decisions is in possession of such information rather than whether the person who has title to the trades is in such possession.', {'entities': [(132, 151, 'PER')]}), ('"(4)The disclosures made under this Chapter shall be maintained by the company, for a minimum period of five years, in such form as may be specified.', {'entities': [(67, 78, 'ORG')]}), ('(a). Every promoter 21[, member of the promoter group] , key managerial personnel and director of every company whose securities are listed on any recognised stock exchange shall disclose his holding of securities

Converting doccano annotations to spacy training data

In [89]:
import json
data = []
f = open('file.json1', 'r')
for line in f:
    data.append(json.loads(line))

In [91]:
#For test data
import json
data_test = []
f = open('file2.json1', 'r')
for line in f:
    data_test.append(json.loads(line))

In [93]:
# data[-1]
te_data = []
for k in data_test:
    label_dict = {}
    label_list = []
    for j in k['labels']:
        label_list.append(tuple(j))
    label_dict['entities'] = label_list
    te_data.append((k['text'], label_dict))
te_data = te_data[:125]
te_data

[('"3A. Unless otherwise provided in these Regulations, an issuer making an offer of municipal debt securities shall satisfy the conditions of these Regulations as on the date of filing of the draft offer document or preliminary placement memorandum with the Board and also as on the date of filing the offer document or placement memorandum with the Board or upon registering the offer document or placement memorandum with the Registrar of Companies, as the case may be.]\\n"',
  {'entities': [(40, 51, 'Legal Doc'),
    (56, 62, 'Person - General'),
    (73, 78, 'Object'),
    (82, 107, 'Assets'),
    (126, 136, 'Object'),
    (146, 157, 'Legal Doc'),
    (161, 182, 'time'),
    (190, 210, 'Object'),
    (214, 246, 'Object'),
    (252, 261, 'Person In charge'),
    (277, 295, 'time'),
    (300, 314, 'Object'),
    (318, 338, 'Object'),
    (344, 353, 'Person In charge'),
    (378, 392, 'Object'),
    (396, 416, 'Object'),
    (422, 448, 'Person In charge')]}),
 ('"9[Eligible issuers.\\n"'

In [96]:
# data[-1]
train_data = []
for k in data:
    label_dict = {}
    label_list = []
    for j in k['labels']:
        label_list.append(tuple(j))
    label_dict['entities'] = label_list
    train_data.append((k['text'], label_dict))
# train_data = train_data.extend(te_data)
train = train_data + te_data
len(train)

248

In [80]:
# train_data = train_data.extend(te_data)
# type(train_data)

NoneType

In [48]:
# text_list_uj = []
# for k in uj_train:
#     text_list_uj.append(k[0])
# my_text_list = []
# for j in train_data:
#     my_text_list.append(j[0])
# # print(Diff(text_list_uj,my_text_list))
# li1 = text_list_uj
# li2 = my_text_list
# len(list(set(li1).intersection(set(li2))))
# for k in uj_train:
#     for j in train_data:
#         if k[0] 
# for k in uj_train:
#     for j in train_data:
#         if j[0] == k[0]:
#             j[1]['entities'].extend(k[1]['entities'])

0

In [6]:
train_data.extend(uj_train)
len(train_data)

NameError: name 'uj_train' is not defined

In [97]:
import random
# print(len(train_data))
# new_data = train_data
random.shuffle(train)
# random.shuffle(new_data)
TRAIN_DATA = train[:190]
te_data = train[190:]
len(te_data)

58

In [98]:
import random
def train_spacy(data,iterations,empty):

    # Disable pipeline components you dont need to change

    
    TRAIN_DATA = data
    if empty == True:
        nlp = spacy.blank('en')  # create blank Language class
        # create the built-in pipeline components and add them to the pipeline
        # nlp.create_pipe works for built-ins that are registered with spaCy
        if 'ner' not in nlp.pipe_names:
            ner = nlp.create_pipe('ner')
            nlp.add_pipe(ner, last=True)
    
    elif empty ==False:
        nlp=spacy.load('en_core_web_sm')  
        ner=nlp.get_pipe("ner")
    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    if empty:
        other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
        with nlp.disable_pipes(*other_pipes):  # only train NER
            optimizer = nlp.begin_training()
            for itn in range(iterations):
                print("Statring iteration " + str(itn))
                random.shuffle(TRAIN_DATA)
                losses = {}
                for text, annotations in TRAIN_DATA:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,  # dropout - make it harder to memorise data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                print(losses)
    else:
        pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
        unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

        with nlp.disable_pipes(*unaffected_pipes):    
            optimizer = nlp.begin_training()
            for itn in range(iterations):
                print("Statring iteration " + str(itn))
                random.shuffle(TRAIN_DATA)
                losses = {}
                for text, annotations in TRAIN_DATA:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,  # dropout - make it harder to memorise data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                print(losses)
    return nlp


prdnlp = train_spacy(TRAIN_DATA, 100,True)

# Save our trained Model
modelfile = input("Enter your Model Name: ")
prdnlp.to_disk(modelfile)

# #Test your text
# test_text = input("Enter your testing text: ")
# doc = prdnlp(test_text)
# for ent in doc.ents:
#     print(ent.text, ent.start_char, ent.end_char, ent.label_)


/usr/local/lib/python3.8/dist-packages/spacy/language.py:635: UserWarning: [W033] Training a new parser or NER using a model with no lexeme normalization table. This may degrade the performance of the model to some degree. If this is intentional or the language you're using doesn't have a normalization table, please ignore this warning. If this is surprising, make sure you have the spacy-lookups-data package installed. The languages with lexeme normalization tables are currently: da, de, el, en, id, lb, pt, ru, sr, ta, th.
  proc.begin_training(
/usr/local/lib/python3.8/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text ""(2) The internal controls shall include the follo..." with entities "[(66, 75, 'Person - General'), (305, 317, 'Object'...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)


Statring iteration 0


/usr/local/lib/python3.8/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text ""(2)Original Information may, at the discretion of..." with entities "[(4, 24, 'Object'), (51, 60, 'Person In charge'), ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/usr/local/lib/python3.8/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text ""(1) Upon collection or substantial recovery of th..." with entities "[(52, 70, 'Assets'), (87, 109, 'Assets'), (111, 12...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/usr/local/lib/python3.8/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned

/usr/local/lib/python3.8/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text ""\u201c(5) The advertisement shall urge the invest..." with entities "[(15, 28, 'Object'), (44, 53, 'Person - General'),...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/usr/local/lib/python3.8/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text ""4.No issuer shall be eligible to issue debt secur..." with entities "[(6, 12, 'Person - General'), (34, 55, 'Assets'), ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/usr/local/lib/python3.8/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned

/usr/local/lib/python3.8/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text ""(2) The issuer and lead manager(s) shall ensure t..." with entities "[(9, 15, 'Person - General'), (20, 32, 'Person - G...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/usr/local/lib/python3.8/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text ""(2)No person shall procure from or cause the comm..." with entities "[(78, 117, 'Object'), (7, 13, 'Person - General'),...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/usr/local/lib/python3.8/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned

/usr/local/lib/python3.8/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text ""Provided that the insider may prove his innocence..." with entities "[(15, 26, 'Person - General'), (72, 85, 'Object'),...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/usr/local/lib/python3.8/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text ""19The words \u201clead merchant banker\u201d subs..." with entities "[(7, 12, 'Object'), (19, 39, 'Person - General'), ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/usr/local/lib/python3.8/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned

/usr/local/lib/python3.8/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text ""7G. No Reward shall be made to an Informant:-\n(1..." with entities "[(5, 14, 'Assets'), (35, 44, 'Person - General'), ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/usr/local/lib/python3.8/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text ""(1) Every person required to have a Code of Condu..." with entities "[(5, 17, 'Person - General'), (37, 52, 'Legal Doc'...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/usr/local/lib/python3.8/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned

/usr/local/lib/python3.8/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text ""(1) Informants who are considered tentatively eli..." with entities "[(5, 15, 'Person - General'), (60, 68, 'Assets'), ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/usr/local/lib/python3.8/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text ""(5)No person shall have the right to compel discl..." with entities "[(4, 13, 'Person - General'), (29, 34, 'Object'), ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/usr/local/lib/python3.8/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned

/usr/local/lib/python3.8/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text ""NOTE: It is intended that since the trading plan ..." with entities "[(37, 49, 'Object'), (73, 85, 'Legal Doc'), (91, 1...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/usr/local/lib/python3.8/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text ""Provided that where the Informant does not submit..." with entities "[(21, 34, 'Person - General'), (55, 92, 'Legal Doc...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/usr/local/lib/python3.8/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned

{'ner': 4598.279057137641}
Statring iteration 1
{'ner': 4198.27924156106}
Statring iteration 2
{'ner': 2575.099177408142}
Statring iteration 3
{'ner': 2450.640898055251}
Statring iteration 4
{'ner': 2198.7536335788373}
Statring iteration 5
{'ner': 2023.0768671258224}
Statring iteration 6
{'ner': 1855.0365561621513}
Statring iteration 7
{'ner': 1722.7819809857954}
Statring iteration 8
{'ner': 1703.5239726515945}
Statring iteration 9
{'ner': 1657.6913792862413}
Statring iteration 10
{'ner': 1723.199901290492}
Statring iteration 11
{'ner': 1583.9798607202304}
Statring iteration 12
{'ner': 1401.1245812697673}
Statring iteration 13
{'ner': 1485.591569795054}
Statring iteration 14
{'ner': 1403.5050086837887}
Statring iteration 15
{'ner': 1389.4911629425917}
Statring iteration 16
{'ner': 1259.8386685433982}
Statring iteration 17
{'ner': 1389.7714056746606}
Statring iteration 18
{'ner': 1271.146570421541}
Statring iteration 19
{'ner': 1329.500375941153}
Statring iteration 20
{'ner': 1223.39495

In [107]:
#Converting training to testing examples:
test_data = []

for k in train_data:
# for k in te_data:
    test_data.append((k[0], k[1]['entities']))
test_data

[('"(1) In these regulations, unless the context otherwise requires, the following words, expressions and derivations therefrom shall have the meanings assigned to them as under:–\\n(a)“Act” means the Securities and Exchange Board of India Act, 1992 (15 of 1992);\\n(b)“Board” means the Securities and Exchange Board of India;\\n(c)“compliance officer” means any senior officer, designated so and reporting to the board of directors or head of the organization in case board is not there, who is financially literate and is capable of appreciating requirements for legal and regulatory compliance under these regulations and who shall be responsible for compliance of policies, procedures, maintenance of records, monitoring adherence to the rules for the preservation of unpublished price sensitive information, monitoring of trades and the implementation of the codes specified in these regulations under the overall supervision of the board of directors of the listed company or the head of an org

In [108]:


# You can find different metrics including F-score, recall and precision in spaCy/scorer.py.

# This example shows how you can use it:

import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer

def evaluate(ner_model, examples):
    scorer = Scorer()
    for input_, annot in examples:
        doc_gold_text = ner_model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot)
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
    return scorer.scores

# example run

# examples = [
#     ('Who is Shaka Khan?',
#      [(7, 17, 'PERSON')]),
#     ('I like London and Berlin.',
#      [(7, 13, 'LOC'), (18, 24, 'LOC')])
# ]
examples = test_data
# print(examples)
ner_model_path = 'my_mixed_random/'
ner_model = spacy.load(ner_model_path) # for spaCy's pretrained use 'en_core_web_sm'
results = evaluate(ner_model, examples)

<ipython-input-108-fa4f46496065>:13: UserWarning: [W030] Some entities could not be aligned in the text ""(1) In these regulations, unless the context othe..." with entities "[(80, 84, 'Object'), (87, 97, 'Object'), (103, 113...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc_gold_text, entities=annot)
<ipython-input-108-fa4f46496065>:13: UserWarning: [W030] Some entities could not be aligned in the text ""(1) No insider shall communicate, provide, or all..." with entities "[(8, 15, 'Person - General'), (328, 345, 'Object')...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc_gold_text, entities=annot)
<ipython-input-108-fa4f46496065>:13: UserWarning: [W030] Some entities could not be aligned in the text ""(2)No person shall procu

<ipython-input-108-fa4f46496065>:13: UserWarning: [W030] Some entities could not be aligned in the text ""Provided that the implementation of the trading p..." with entities "[(41, 53, 'Object'), (84, 123, 'Object'), (141, 15...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc_gold_text, entities=annot)
<ipython-input-108-fa4f46496065>:13: UserWarning: [W030] Some entities could not be aligned in the text ""NOTE: It is intended that since the trading plan ..." with entities "[(37, 49, 'Object'), (73, 85, 'Legal Doc'), (91, 1...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc_gold_text, entities=annot)
<ipython-input-108-fa4f46496065>:13: UserWarning: [W030] Some entities could not be aligned in the text ""(1) Every public disclos

<ipython-input-108-fa4f46496065>:13: UserWarning: [W030] Some entities could not be aligned in the text ""(5)No person shall have the right to compel discl..." with entities "[(4, 13, 'Person - General'), (29, 34, 'Object'), ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc_gold_text, entities=annot)
<ipython-input-108-fa4f46496065>:13: UserWarning: [W030] Some entities could not be aligned in the text ""Explanation 1. – The confidentiality in respect o..." with entities "[(56, 64, 'Object'), (86, 95, 'Person - General'),...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc_gold_text, entities=annot)
<ipython-input-108-fa4f46496065>:13: UserWarning: [W030] Some entities could not be aligned in the text ""(1) Every person require

<ipython-input-108-fa4f46496065>:13: UserWarning: [W030] Some entities could not be aligned in the text ""(1) The Chief Executive Officer, Managing Directo..." with entities "[(281, 296, 'Object'), (5, 32, 'Person In charge')...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc_gold_text, entities=annot)
<ipython-input-108-fa4f46496065>:13: UserWarning: [W030] Some entities could not be aligned in the text ""(2) The internal controls shall include the follo..." with entities "[(66, 75, 'Person - General'), (305, 317, 'Object'...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc_gold_text, entities=annot)
<ipython-input-108-fa4f46496065>:13: UserWarning: [W030] Some entities could not be aligned in the text ""(3)The board of director

In [109]:
results

{'uas': 0.0,
 'las': 0.0,
 'las_per_type': {'': {'p': 0.0, 'r': 0.0, 'f': 0.0}},
 'ents_p': 94.9579831932773,
 'ents_r': 93.7759336099585,
 'ents_f': 94.36325678496868,
 'ents_per_type': {'Person In charge': {'p': 97.61904761904762,
   'r': 91.11111111111111,
   'f': 94.25287356321839},
  'Legal Doc': {'p': 96.22641509433963,
   'r': 96.22641509433963,
   'f': 96.22641509433963},
  'Company': {'p': 87.5, 'r': 87.5, 'f': 87.5},
  'Object': {'p': 95.83333333333334,
   'r': 95.83333333333334,
   'f': 95.83333333333334},
  'Person - General': {'p': 92.10526315789474,
   'r': 97.22222222222221,
   'f': 94.59459459459458},
  'Assets': {'p': 92.3076923076923,
   'r': 92.3076923076923,
   'f': 92.3076923076923},
  'time': {'p': 85.71428571428571,
   'r': 85.71428571428571,
   'f': 85.71428571428571},
  'Penalty': {'p': 100.0, 'r': 100.0, 'f': 100.0},
  'Investigation': {'p': 100.0, 'r': 75.0, 'f': 85.71428571428571},
  'Place': {'p': 0.0, 'r': 0.0, 'f': 0.0}},
 'tags_acc': 0.0,
 'token_acc': 1

In [110]:
output_dir = 'my_mixed_random/'
nlp2 = spacy.load(output_dir)
# assert nlp2.get_pipe("ner").move_names == move_names
# doc2 = nlp2('Any term in an agreement (oral or written) or Code of Conduct, is void in so far as it purports to preclude any person, other than an advocate, from submitting to the Board information relating to the violation of the securities laws that has occurred, is occurring or has a reasonable belief that it would occur.')
# doc2 = nlp2('An Informant who may be liable for enforcement action by the Board based on his or her conduct in connection with securities laws violations reported in the Voluntary Information Disclosure Form filed with the Board, may simultaneously or at any time thereafter file an application seeking settlement with confidentiality under Chapter IX of the Securities and Exchange Board of India (Settlement Proceedings) Regulations, 2018')
# doc2 = nlp2('No person shall have the right to compel disclosure of the identity, existence of an Informant or the information provided by an Informant, except to the extent relied upon in any proceeding initiated against such person by the Board.')
# doc2 = nlp2('Upon collection or substantial recovery of the monetary sanctions amounting to at least twice the Reward, the Board may at its sole discretion, declare an Informant eligible for Reward and intimate the Informant or his or her legal representative to file an application in the format provided in Schedule-E for claiming such Reward:')
# test_text = input("Enter your testing text: ")
# doc2 = nlp2('his provision is intended to mandate persons other than listed companies and market intermediaries that are required to handle unpublished price sensitive information to formulate a code of conduct governing trading in securities by their employees. These entities include professional firms such as auditors, accountancy firms, law firms, analysts, consultants etc., assisting or advising listed companies, market intermediaries and other capital market participants. Even entities that normally operate outside the capital market may handle unpublished price sensitive information. This provision would mandate all of them to formulate a code of conduct.')
# doc2 = nlp2('Notwithstanding any action taken by the Board against an Informant, the Informant may, after payment of any monetary amounts be eligible for a Reward')
# doc2 = nlp2('In case there are multiple lead managers, at least one lead manager to the issue shall not be an associate (as defined under the Securities and Exchange Board of India (Merchant Bankers) Regulations, 1992) of the issuer')
# doc2 = nlp2(' No issuer shall make a public issue of municipal debt securities unless a draft offer document has been filed with the Board and designated stock exchange through the lead manager')
doc2 = nlp2('a permanent employee of the company who has been working in India or outside India; or a director of the company, whether a whole time director or not but excluding an independent director')
for ent in doc2.ents:
  print(ent.label_, ent.text)

Person - General employee
Company the company
Object in India
Person In charge outside India
Person In charge a director of the company
time whole time director
Investigation independent


In [29]:
test_text = input("Enter your testing text: ")
doc = prdnlp(test_text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


Enter your testing text: his provision is intended to mandate persons other than listed companies and market intermediaries that are required to handle unpublished price sensitive information to formulate a code of conduct governing trading in securities by their employees. These entities include professional firms such as auditors, accountancy firms, law firms, analysts, consultants etc., assisting or advising listed companies, market intermediaries and other capital market participants. Even entities that normally operate outside the capital market may handle unpublished price sensitive information. This provision would mandate all of them to formulate a code of conduct.


In [105]:
test_data[8]

('"(8)The issuer filing a shelf prospectus shall file a copy of an information memorandum with the recognised stock exchanges and the Board and in case of a corporate municipal entity, file the same with the Registrar of Companies.\\n"',
 [(8, 14, 'Person - General'),
  (24, 40, 'Legal Doc'),
  (65, 87, 'Object'),
  (97, 123, 'Person In charge'),
  (128, 137, 'Person In charge'),
  (153, 181, 'Person In charge'),
  (202, 228, 'Person In charge')])